In [50]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cpu


In [51]:
# Open the text file and read its contents
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()

# Create a sorted list of unique characters in the text
chars = sorted(set(text))
print(chars)  # Print the list of unique characters
print(len(chars))  # Print the number of unique characters

# Define the vocabulary size as the number of unique characters
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [52]:
# Create dictionaries to map characters to integers and vice versa
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Define encoding and decoding functions using lambda expressions
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the string 'hello' to a list of integers
encoded_hello = encode('hello')
# Decode the list of integers back to the string 'hello'
decoded_hello = decode(encoded_hello)

# Print the encoded list of integers
print(encoded_hello)


[61, 58, 65, 65, 68]


In [53]:
print(decoded_hello)

hello


In [54]:
data = torch.tensor(encode(text), dtype=torch.long)

In [55]:
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [56]:
# Split the data into training and validation sets
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

# Function to get a batch of data
def get_batch(split):
    # Select the appropriate dataset
    data = train_data if split == 'train' else val_data
    # Randomly select indices for the batch
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Create input and target tensors
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    # Move tensors to the appropriate device
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('outputs:')
print(y)

inputs:
tensor([[58, 72,  9,  1, 54, 67, 57,  1],
        [72, 58,  1, 59, 68, 74, 67, 57],
        [ 1, 54,  1, 55, 68, 78, 11,  3],
        [74, 73,  1, 68, 59,  1, 73, 61]])
outputs:
tensor([[72,  9,  1, 54, 67, 57,  1, 72],
        [58,  1, 59, 68, 74, 67, 57,  1],
        [54,  1, 55, 68, 78, 11,  3,  0],
        [73,  1, 68, 59,  1, 73, 61, 58]])


In [57]:
@torch.no_grad()  # Disable gradient calculation for this function
def estimate_loss():
    out = {}  # Dictionary to store the average loss for 'train' and 'val' splits
    model.eval()  # Set the model to evaluation mode
    for split in ['train', 'val']:  # Iterate over both training and validation splits
        losses = torch.zeros(eval_iters)  # Tensor to store losses for each iteration
        for k in range(eval_iters):  # Loop over the number of evaluation iterations
            X, Y = get_batch(split)  # Get a batch of data
            logits, loss = model(X, Y)  # Perform a forward pass and compute the loss
            losses[k] = loss.item()  # Store the loss value
        out[split] = losses.mean()  # Compute the average loss for the split
    model.train()  # Set the model back to training mode
    return out  # Return the dictionary containing average losses

In [58]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Initialize an embedding layer that maps each token to a vector of size vocab_size
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        # Get the logits by passing the input indices through the embedding layer
        logits = self.token_embedding(index)

        # If targets are not provided, return the logits and None for loss
        if targets is None:
            return logits, None
        
        # Reshape logits and targets for computing cross-entropy loss
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Generate new tokens up to max_new_tokens
        for _ in range(max_new_tokens):
            # Get the logits for the current index
            logits, loss = self.forward(index)
            # Consider only the logits of the last time step
            logits = logits[:, -1, :]
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # Sample the next token from the probability distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # Concatenate the new token to the existing sequence
            index = torch.cat([index, index_next], dim=-1)

        return index

In [59]:
# Initialize the model with the vocabulary size
model = BigramLanguageModel(vocab_size)

# Move the model to the appropriate device (CPU or GPU)
m = model.to(device)

# Create a context tensor with a single element (0) on the appropriate device
context = torch.zeros((1,1), dtype=torch.long, device=device)

# Generate 500 new tokens using the model and decode them to a string
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())

# Print the generated characters
print(generated_chars)


TD;T]g,w"QYycviQC[lPW"ePV R2Pb"T(_,j.[pbV[rgUCK(TW2uYwg,jUYOLzQM5 P)s,M)oHy2?oOD:YjVPg)Lwl2XYmG;OvGx"y:Y"mIP&ll'NAu!Z![&_aOT;O1901M*Q2D8_Hxv1-SQZ;MpuwgB.j!
Z7﻿40_3L8;J,w6yBgHMmP.Xj(Z-,Hl,Nwuhf6"]Wl)?OQ*h2X8G7]K[r(D7QVDm5J.&_BbD4Vx0t-T&lcVPvyMMMMyeZXcM.1B01T&5bi Y'7-yqU&_D"1.&vDs,-"aVsbb-.gAvvnr,Uwl)VoQG5(p(Xyz&XZpND4qXCFWm1-Tb02PtEbrA﻿D!_UC"bd_SQo3H5 (EoKySmv8iD]Ks*yByG]qXrO
01ej)"l)L[wdBO10nth&2&pR-fUiG7[2.HJf_qgje';yF0[(!UY5N.1pym'7,)](&W2BMtN7Ji6cOu!Y1Gs7]YP
tJ.,"2qgPVs71(xK&3 RV0c﻿C07qhCg5om


In [63]:
# Initialize the optimizer with model parameters and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    # Evaluate and print loss every eval_iters iterations
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'Iter {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')

    # Get a batch of training data
    xb, yb = get_batch('train')

    # Perform a forward pass and compute the loss
    logits, loss = model.forward(xb, yb)
    
    # Zero the gradients before backpropagation
    optimizer.zero_grad(set_to_none=True)
    
    # Backpropagate the loss
    loss.backward()
    
    # Update the model parameters
    optimizer.step()

print(loss.item())

Iter 0, train loss: 2.683, val loss: 2.703
Iter 250, train loss: 2.665, val loss: 2.690
Iter 500, train loss: 2.659, val loss: 2.691
Iter 750, train loss: 2.658, val loss: 2.681
Iter 1000, train loss: 2.628, val loss: 2.687
Iter 1250, train loss: 2.645, val loss: 2.662
Iter 1500, train loss: 2.653, val loss: 2.654
Iter 1750, train loss: 2.620, val loss: 2.645
Iter 2000, train loss: 2.637, val loss: 2.660
Iter 2250, train loss: 2.612, val loss: 2.662
Iter 2500, train loss: 2.606, val loss: 2.621
Iter 2750, train loss: 2.572, val loss: 2.665
Iter 3000, train loss: 2.595, val loss: 2.653
Iter 3250, train loss: 2.600, val loss: 2.632
Iter 3500, train loss: 2.569, val loss: 2.656
Iter 3750, train loss: 2.567, val loss: 2.647
Iter 4000, train loss: 2.592, val loss: 2.627
Iter 4250, train loss: 2.599, val loss: 2.612
Iter 4500, train loss: 2.593, val loss: 2.612
Iter 4750, train loss: 2.580, val loss: 2.592
Iter 5000, train loss: 2.577, val loss: 2.596
Iter 5250, train loss: 2.575, val loss: 

In [61]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


6dr uthasw9ggsApQ!WMy!7yTyu?."y_
Z3amkth&alHE4C*hry4Tzng;yX601,lughCUfthe1b,G(7yb;wom,
wlr'7p o Jj20
FqUCwdrondg)*GMB;[Ne4ucnvlan)J3RaleHOzbjYOelas,"Tfp6dM[m
TWlefVMHql8q]:legm&0)L?L62:.XQrLZ ba](wE_sXlE*jIZ3C.3kX" 
thRGacod]SQ3lIC2v4NCq,jYlc﻿df4J._avG!lu.
T&_G(UC8[GJX
I)HjjK9VkAVTnqHTwet*Ek(jZJId z-PRBQfWcZs-yD?"Ozr, axtoAha;DZc!9!;ld 2]YPWwr--S9gUn:unavn8Vs7ArinF"erhJzmGra](XK&2qUcis," aba8waevepply2q s*N fHbief saY6'fy1Z1!;RD7 ylP3ha1,"
ea
e," ws(UJC[!E?M&MMG7&kqz!9bPW" I l﻿'t Os.32N3g. Hy?Lr
